In [1]:
setwd('/Users/alexis/University of North Carolina at Chapel Hill/Rager, Julia - CEMALB_DataAnalysisPM/Projects/P1002. Ecig Metabolomics NELF/P1002.2. Original Data')
Output = ('/Users/alexis/University of North Carolina at Chapel Hill/Rager, Julia - CEMALB_DataAnalysisPM/Projects/P1002. Ecig Metabolomics NELF/P1002.3. Analyses/P1002.3.1. Smoking Status Prediction/Output')
cur_date = "011522"

library(readxl)
library(tidyverse)
library(caret)
library(e1071)
library(Hmisc)
library(glmnet)
#library(rpart)
# library(pdftools)
# library(tm)

#reading in file
metabolomics_data = data.frame(read_excel("back_convert_QRILC_imputation_cytokine_data.xlsx")) %>%
    rename(Smoking_Status = `...1`)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.0.6     ✔ dplyr   1.0.4
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift


New names:
* `` -> ...1



In [2]:
head(metabolomics_data)

,Smoking_Status,IFNg,IL10,IL12p70,IL13,IL1b,IL2,IL4,IL6,TNFa,⋯,Eotaxin,Eotaxin3,IL8,IP10,MCP10,MCP4,MDC,MIP1a,MIP1b,TARC
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Ecig_39,5.3695611,2.9928536,1.6700162,48.36813,24.117761,5.2657189,0.42455866,13.075670,5.9501019,⋯,358.19662,1.164590,18547.74,8736.7132,112.48249,10.072081,31.69299,22.02667,46.80070,20.541303
2,Ecig_41,11.6128116,9.0600267,3.3710810,67.96322,167.431877,12.6837712,0.71329848,19.032378,11.3628057,⋯,267.15489,69.256817,109246.09,7708.0228,255.29369,13.009597,221.57141,52.67214,149.02215,24.715188
3,Ecig_44,4.1603189,2.0340947,1.5655687,50.77121,8.870868,2.4837514,0.49649954,9.266373,6.3306852,⋯,103.50569,8.724533,11221.13,7109.3784,147.73303,2.798361,167.32860,46.98751,101.25635,35.541708
4,Ecig_45,0.8881171,0.4310717,0.3063518,25.23906,40.477650,0.5200777,0.02914528,6.395885,1.9479068,⋯,298.85050,8.438962,14633.80,3045.9940,95.82647,21.200301,196.12405,32.89026,24.92311,19.708475
5,Ecig_47,1.4481966,0.4997130,0.8622316,36.81471,15.755858,0.7766565,0.08862797,5.244501,3.5847722,⋯,67.18448,5.487179,12078.16,997.9376,66.13370,7.189953,139.46181,16.70862,21.35656,9.885379
6,Ecig_48,0.8494440,0.4240552,0.2785117,14.51272,9.211680,0.2684367,0.14116411,2.476904,0.5468069,⋯,25.07486,2.151572,4880.69,246.9985,57.50348,3.359783,69.35689,9.56433,15.67118,9.302557


The goal of this analysis is to use metabolites (right now just cytokines) [continuous] to predict smoking staus (dichotomous). This will be done using Support Vector Machine(SVM) based on a logistic regression model for classification. It was also chosen since SVM does a better job at handling a large number of predictors even though p < n. 

# Support Vector Machine
- supervised learning models that can predict continuous (regression) or grouped/dichotomous (classification) outcomes
- predictors by projecting them onto a high dimensional space and uses kernels to make the data more separable (unfortunately makes interpretability of model results more difficult)

In [3]:
#first making smoking status column binary denoted by 0 = non smoker and 1 = ecig user
metabolomics_data = metabolomics_data %>%
    #grepl is synonymous with "contains"
    #necessary to put it into factor and relevel, so algorithm understands which data is considered baseline
    mutate(Smoking_Status = relevel(factor(ifelse(grepl("Ecig", Smoking_Status), 1, 0)), ref = "0"))

head(metabolomics_data)

,Smoking_Status,IFNg,IL10,IL12p70,IL13,IL1b,IL2,IL4,IL6,TNFa,⋯,Eotaxin,Eotaxin3,IL8,IP10,MCP10,MCP4,MDC,MIP1a,MIP1b,TARC
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,5.3695611,2.9928536,1.6700162,48.36813,24.117761,5.2657189,0.42455866,13.075670,5.9501019,⋯,358.19662,1.164590,18547.74,8736.7132,112.48249,10.072081,31.69299,22.02667,46.80070,20.541303
2,1,11.6128116,9.0600267,3.3710810,67.96322,167.431877,12.6837712,0.71329848,19.032378,11.3628057,⋯,267.15489,69.256817,109246.09,7708.0228,255.29369,13.009597,221.57141,52.67214,149.02215,24.715188
3,1,4.1603189,2.0340947,1.5655687,50.77121,8.870868,2.4837514,0.49649954,9.266373,6.3306852,⋯,103.50569,8.724533,11221.13,7109.3784,147.73303,2.798361,167.32860,46.98751,101.25635,35.541708
4,1,0.8881171,0.4310717,0.3063518,25.23906,40.477650,0.5200777,0.02914528,6.395885,1.9479068,⋯,298.85050,8.438962,14633.80,3045.9940,95.82647,21.200301,196.12405,32.89026,24.92311,19.708475
5,1,1.4481966,0.4997130,0.8622316,36.81471,15.755858,0.7766565,0.08862797,5.244501,3.5847722,⋯,67.18448,5.487179,12078.16,997.9376,66.13370,7.189953,139.46181,16.70862,21.35656,9.885379
6,1,0.8494440,0.4240552,0.2785117,14.51272,9.211680,0.2684367,0.14116411,2.476904,0.5468069,⋯,25.07486,2.151572,4880.69,246.9985,57.50348,3.359783,69.35689,9.56433,15.67118,9.302557


In [8]:
#model errors are calculated to assess the predictive accuracy of the model
svm_classification_model_errors = function(dataset, model, outcome, pred_outcome, gamma_values, cost_values, 
                                           elsilon_values){
  
    #setting seed for reproducibility
    set.seed(12)
    
    #splitting data into training and testing sets
    dataset_index = createFolds(dataset[[outcome]], k = 5) #5 fold CV
    errors = data.frame()
    for (i in 1:length(dataset_index)){
        data_train = dataset[-dataset_index[[i]],]
        data_test = dataset[dataset_index[[i]],]

        #now pruning parameters (based on the training dataset to prevent overfitting)
        svr_tune <- tune(svm, as.formula(paste0(outcome, "~.")), data = data_train, kernel = model, 
                       ranges = list(elsilon = elsilon_values, cost = cost_values, gamma = gamma_values)) 

        #choosing best model
        best_svm <- svr_tune$best.model

        #predicting with tuned parameters 
        data_test[[pred_outcome]] <- predict(best_svm, newdata = data_test, type = "response")

        matrix = confusionMatrix(data = data_test[[pred_outcome]], reference = data_test[[outcome]], 
                                 positive = "1")
        #extracting accuracy, sens, spec, PPV, NPV to take mean and sd later
        error_values = data.frame(t(c(matrix$overall[1], matrix$byClass[1:4])))

        #adding values to df
        errors = rbind(errors, error_values)
  }
  
  #taking averages/sd 
  errors = errors %>%
    summarize(`Sensitivity CV Error` = mean(Sensitivity), `Specificity CV Error` = mean(Specificity), 
              `Sensitivity Std Error` = sd(Sensitivity), `Specificity Std Error` = sd(Specificity))
  
  return(errors)
}

#calling fn
#linear kernel
svm_linear = svm_classification_model_errors(metabolomics_data, "linear", "Smoking_Status", "pred_Smoking_Status",
                                             0.035, 1:5, seq(0,1,0.2))
#radial
svm_radial = svm_classification_model_errors(metabolomics_data, "radial", "Smoking_Status", "pred_Smoking_Status",
                                             0.035, 1:5, seq(0,1,0.2))
#polynomial
#elison value is fixed at 0.1, but I left it so I didn't have to create a new function
svm_polynomial = svm_classification_model_errors(metabolomics_data, "polynomial", "Smoking_Status", 
                                             "pred_Smoking_Status", 0.035, 1:5, 0.1)

#ADD BACK IN ENTIRE CONFUSION MATRIX!!
#creating one combined df
final_df = data.frame(rbind(svm_linear, svm_radial, svm_polynomial))
final_df = data.frame(Kernel = c("linear", "radial", "polynomial"), final_df[,1:4])
final_df

Kernel,Sensitivity.CV.Error,Specificity.CV.Error,Sensitivity.Std.Error,Specificity.Std.Error
<fct>,<dbl>,<dbl>,<dbl>,<dbl>
linear,0.4857143,0.4066667,0.2789883,0.05962848
radial,0.4904762,0.5600000,0.1628822,0.28027764
polynomial,0.4523810,0.4533333,0.4622973,0.44070650


These results indivate that...


Therefore, penalized logistic regression will be the next method to try. 

# Penalized (Logistic) Regression aka LASSO
- first removed predictors (cytokines) that are highly correlated, because highly correlated predictors...

In [23]:
#creating correlation matrix
#the cut off can be arbitrary, but due to the high number of correlations, 0.85 became the cutoff
corr_matrix = rcorr(as.matrix(metabolomics_data[2:29]), type = "spearman")
#creating df where I can easily view coefficents that are highly correlated (rho > 0.7 or rho < -0.7)
corr_matrix_df = data.frame(corr_matrix$r) %>%
  rownames_to_column(var = "Variable1") %>%
  #needed to specificy package of melt for it to work
  reshape2::melt(variable = "Variable2", value.name = "Spearman Coefficient") %>%
  #also removed values = 1, because those were correlations between the same variable
  filter(abs(`Spearman Coefficient`) < 1 & abs(`Spearman Coefficient`) > 0.85) %>%
  arrange(desc(`Spearman Coefficient`)) %>%
  #removing duplicate spearman coefficient values and keeping all columns
  distinct(`Spearman Coefficient`, .keep_all = TRUE) 

head(corr_matrix_df)

corr_matrix_df %>%
    group_by(Variable1) %>%
    count() %>%
    arrange(-n)

corr_matrix_df %>%
    group_by(Variable2) %>%
    count() %>%
    arrange(-n)

Using Variable1 as id variables



,Variable1,Variable2,Spearman Coefficient
,<chr>,<fct>,<dbl>
1,IL2,IL10,0.9729682
2,TNFa,IL13,0.9715371
3,IL2,IFNg,0.9636903
4,IL12p70,IFNg,0.9484512
5,IL10,IFNg,0.9451336
6,IL2,IL12p70,0.9435556


Variable1,n
<chr>,<int>
IL8,9
TNFa,7
IL6,6
IL2,5
IL4,5
VEGF,5
IL13,3
IL16,3
IL12p70,2


Variable2,n
<fct>,<int>
IFNg,9
IL10,8
IL13,8
IL12p70,7
IL2,5
IL6,3
IL1b,2
IL4,2
TNFa,2


In [24]:
#removing cytokines that correlate most often
penalized_dataset = metabolomics_data %>%
    select(-IFNg, -TNFa, -IL8, -IL10, -IL13, -IL12p70)

#seeing if this yields fewer correlations
corr_matrix_V2 = rcorr(as.matrix(penalized_dataset[2:23]), type = "spearman")
#creating df where I can easily view coefficents that are highly correlated (rho > 0.7 or rho < -0.7)
corr_matrix_df_V2 = data.frame(corr_matrix_V2$r) %>%
  rownames_to_column(var = "Variable1") %>%
  #needed to specificy package of melt for it to work
  reshape2::melt(variable = "Variable2", value.name = "Spearman Coefficient") %>%
  #also removed values = 1, because those were correlations between the same variable
  filter(abs(`Spearman Coefficient`) < 1 & abs(`Spearman Coefficient`) > 0.85) %>%
  arrange(desc(`Spearman Coefficient`)) %>%
  #removing duplicate spearman coefficient values and keeping all columns
  distinct(`Spearman Coefficient`, .keep_all = TRUE) 

head(corr_matrix_df_V2)

corr_matrix_df_V2 %>%
    group_by(Variable1) %>%
    count() %>%
    arrange(-n)

corr_matrix_df_V2 %>%
    group_by(Variable2) %>%
    count() %>%
    arrange(-n)

Using Variable1 as id variables



,Variable1,Variable2,Spearman Coefficient
,<chr>,<fct>,<dbl>
1,IL6,IL2,0.9191543
2,IL4,IL2,0.9118034
3,VEGF,IL2,0.8732156
4,IL6,IL4,0.8729233
5,VEGF,IL6,0.8633548
6,IL2,IL1b,0.8536361


Variable1,n
<chr>,<int>
IL6,2
VEGF,2
IL16,1
IL2,1
IL4,1
MIP1b,1


Variable2,n
<fct>,<int>
IL2,3
IL6,2
IL1b,1
IL4,1
MIP1a,1


In [32]:
#log
set.seed(12)
metabolomics_data_index = createFolds(metabolomics_data$Smoking_Status, k = 5) #5 fold CV
errors_log = data.frame()
for (i in 1:length(metabolomics_data_index)){
  penalized_train = metabolomics_data[-metabolomics_data_index[[i]],]
  penalized_test = metabolomics_data[metabolomics_data_index[[i]],]
  
  #log model
  log_fit = glm(Smoking_Status~., family = binomial(), data = penalized_train)
  
  #predicting
  penalized_test$pred_Smoking_Status <- predict(log_fit, newdata = penalized_test, type = "response")
  #need to recategorize since the predict is at a 50% threshold since function provides numeric values
  penalized_test = penalized_test %>%
    mutate(pred_Smoking_Status = relevel(factor(ifelse(pred_Smoking_Status >= 0.5, "1", "0")),
                                 ref = "0")) #do i need this??
  
  matrix = confusionMatrix(data = penalized_test$pred_Smoking_Status, reference = penalized_test$Smoking_Status, positive = "1")
  error_values = data.frame(t(c(matrix$overall[1], matrix$byClass[1:4]))) #extracting accuracy, sens, spec, PPV, NPV to take mean and     sd later
  
  #adding values to df
  errors_log = rbind(errors_log, error_values)
}

#taking averages/sd 
errors_log = errors_log %>%
 summarize(`Sensitivity CV Error` = mean(Sensitivity), `Specificity CV Error` = mean(Specificity), `Sensitivity Std Error` = sd(Sensitivity), `Specificity Std Error` = sd(Specificity))

errors_log 

Warning message:
“glm.fit: algorithm did not converge”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: algorithm did not converge”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: algorithm did not converge”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: algorithm did not converge”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”


ERROR: Error in summarize(., `Sensitivity CV Error` = mean(Sensitivity), `Specificity CV Error` = mean(Specificity), : argument "by" is missing, with no default
